Read recordio to pd.DataFrame and TF.example

In [1]:
#!pip install recordio-0.1.7-cp35-cp35m-macosx.tar.gz

In [2]:
# from google3.file.recordio.python import recordio
import recordio
# we may need to import the external recordio package here, instead of the google internal ones
import tensorflow as tf
import collections
import pandas as pd

def recordio_to_tf_example(recordio_path):
  tf_example_list = []
  with recordio.RecordReader(recordio_path) as rr:
    for index, record in enumerate(rr):
      tf_example = tf.train.SequenceExample()
      tf_example.ParseFromString(record)
      tf_example_list.append(tf_example)
  return tf_example_list

def _tf_feature_value(tf_feature: tf.train.Feature):
  """Converts tf feature value as List.

  Args:
    tf_feature: input feature to be converted.

  Returns:
    A list contains feature value.
  """
  kind = tf_feature.WhichOneof('kind')
  if kind == 'float_list':
    return tf_feature.float_list.value
  if kind == 'int64_list':
    return tf_feature.int64_list.value
  if kind == 'bytes_list':
    return tf_feature.bytes_list.value
  raise ValueError('There is no value for the feature.')

def recordio_to_dataframe(recordio_path):
  tf_example_list = recordio_to_tf_example(recordio_path)
  tf_example_dfs = []
  for tf_example in tf_example_list:
    ret_dict = collections.defaultdict(list)
    for tf_feature_key in tf_example.context.feature:
      ret_dict[tf_feature_key].append(_tf_feature_value(tf_example.context.feature[tf_feature_key]))
    tf_example_dfs.append(pd.DataFrame(ret_dict))
  return tf_example_dfs

OSError: /usr/local/miniconda3/envs/cs330/lib/python3.10/site-packages/recordio/librecordio.so: invalid ELF header